 Graniczna Analiza Danych — Raport

## Autorzy
- Daniel Zdancewicz [145317]
- Alex Terentowicz [145419]


## Opis zbiorów danych

Zbiór danych zawierający informacje o lotniskach

[Wejścia](./resources/datasets/inputs.csv):
- i1: roczna przepustowość terminalu zdefiniowana jako przepływ pasażerów, który port lotniczy może obsłużyć bez poważnych niedogodności (w milionach pasażerów rocznie)
- i2: maksymalna przepustowość zdefiniowana jako średnia liczba operacji (przylotów i/lub odlotów), które można wykonać na pasach startowych portu lotniczego (w liczbie operacji na godzinę)
- i3: przepustowość płyty postojowej lotniska definiowana jako średnia liczba samolotów, które może obsłużyć lotnisko (w liczbie samolotów na godzinę)
- i4: obszar ciążenia lotniska zdefiniowany jako liczba mieszkańców mieszkających w promieniu 100 km od lotniska (w mln mieszkańców)

[Wyjścia](./resources/datasets/outputs.csv):
- o1: ruch pasażerski mierzony całkowitą liczbą pasażerów obsłużonych przez port (w mln pasażerów rocznie)
- o2: liczba operacji statku powietrznego (jeden całkowity ruch to lądowanie lub start statku powietrznego; w tysiącach ruchów rocznie)


[Próbki](./resources/datasets/samples.csv):
- i1: waga wejścia i1
- i2: waga wejścia i2
- i3: waga wejścia i3
- i4: waga wejścia i4
- o1: waga wyjścia i1
- o2: waga wyjścia i2


### Odczyt danych

In [98]:
import pandas as pd
from pandas import DataFrame, read_csv


def load(path: str, *, index_col: str = None) -> DataFrame:
  return read_csv(path, sep=';', index_col=index_col)


inputs_df = load('./resources/datasets/inputs.csv').rename(columns={'Unnamed: 0': 'airport'}).set_index('airport')
outputs_df = load('./resources/datasets/outputs.csv').rename(columns={'Unnamed: 0': 'airport'}).set_index('airport')
samples_df = load('./resources/datasets/samples.csv', index_col='sample')
K = len(inputs_df)


In [99]:
inputs_df.head()

,i1,i2,i3,i4
airport,,,,
WAW,10.5,36,129.4,7.0
KRK,3.1,19,31.6,7.9
KAT,3.6,32,57.4,10.5
WRO,1.5,12,18.0,3.0
POZ,1.5,10,24.0,4.0


In [100]:
outputs_df.head()

,o1,o2
airport,,
WAW,9.5,129.7
KRK,2.9,31.3
KAT,2.4,21.1
WRO,1.5,18.8
POZ,1.3,16.2


In [101]:
samples_df.head()

,i1,i2,i3,i4,o1,o2
sample,,,,,,
1,0.096443,0.059350,0.065224,0.778983,0.620824,0.379176
2,0.032029,0.304959,0.427426,0.235586,0.513536,0.486464
3,0.266256,0.246940,0.217498,0.269306,0.247490,0.752510
4,0.159062,0.635512,0.202159,0.003267,0.223838,0.776162
5,0.096716,0.809025,0.026741,0.067518,0.136140,0.863860


## Efektywność

Do wyznaczenia efektywności został utworzony model analizy danych **CCR** zorientowany na nakłady ( wejścia ).

\begin{align*}
\max & \sum_{n=1}^{N}\mu_n\cdot y_{no} & \\
s.t. & \sum\limits_{m=1}^{M}\nu_m\cdot x_{mo}=1  &  \\
 & \sum\limits_{n=1}^{N}\mu_n\cdot y_{nk}\leq\sum\limits_{m=1}^{M}\nu_n\cdot x_{mk}, & k=1,2,...,K \\
 & \mu_n\nu_m\geq0, & m=1,2,...,M\;, n=1,2,...,N \\
\end{align*}



In [102]:
import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, LpMinimize


def create_model(alternative: int, inputs_df: DataFrame, outputs_df: DataFrame):
  N, M, K = map(len, [outputs_df.columns, inputs_df.columns, inputs_df])
  o = alternative

  model = LpProblem("CCR-input", LpMaximize)

  input_weights = LpVariable.dicts("input_weight", range(M), 0, None, cat='Continuous')
  output_weights = LpVariable.dicts("output_weight", range(N), 0, None, cat='Continuous')

  inputs = inputs_df.values.T
  outputs = outputs_df.values.T

  model.setObjective(sum(
    output_weights[n] * outputs[n][o] for n in range(N)
  ))

  model.addConstraint(sum(
    input_weights[m] * inputs[m][o] for m in range(M)
  ) == 1)

  for k in range(K):
    model.addConstraint(
      sum(output_weights[n] * outputs[n][k] for n in range(N))
      <=
      sum(input_weights[m] * inputs[m][k] for m in range(M))
    )

  return model


def find_efficiency(alternative: int, inputs_df: DataFrame, outputs_df: DataFrame):
  model = create_model(alternative, inputs_df, outputs_df)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  return model.objective.value()


def find_efficiency_weights(alternative: int, inputs_df: DataFrame, outputs_df: DataFrame):
  model = create_model(alternative, inputs_df, outputs_df)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  return model.variablesDict()


efficiency_results = DataFrame(
  [find_efficiency(k, inputs_df, outputs_df) for k in range(K)],
  index=inputs_df.index,
  columns=['efficiency']
).round(3)

efficiency_results

,efficiency
airport,
WAW,1.000
KRK,1.000
KAT,0.591
WRO,1.000
POZ,0.800
LCJ,0.300
GDN,1.000
SZZ,0.271
BZG,1.000


Lotniska efektywne to: Warszawa, Kraków, Wrocław, Gdynia, Bydgoszcz.
LOtniska nieefektywne to: Katowice, Poznań, Łódź, Szczecin, Rzeszów, Zielona Góra.

## Hipotetyczna jednostka porównawcza oraz potrzebne poprawki

Dla każdej z odnalezionych jednostek nieefektywnych odnaleziono hipotetyczną jednostkę porównawczą oraz poprawki potrzebne do osiągnięcia efektywności.


\begin{align*}
\min & \theta & \\
s.t. & \sum\limits_{k=1}^{K}\lambda_k\cdot x_{nk}\leq\theta\cdot x_{no}  & n=1,2,...,N  \\
 & \sum\limits_{k=1}^{K}\lambda_k\cdot y_{nk}\geq y_{mo}, & m=1,2,...,M \\
 & \theta\geq0, &  \\
 & \lambda_k\geq0, & k=1,2,...,K \\
\end{align*}


In [126]:
def create_model(alternative: int, inputs_df: DataFrame, outputs_df: DataFrame):
  N, M, K = map(len, [outputs_df.columns, inputs_df.columns, inputs_df])
  o = alternative
  inputs = inputs_df.values.T
  outputs = outputs_df.values.T

  model = LpProblem("CCR-input", LpMinimize)

  theta = LpVariable("theta", 0, None, cat='Continuous')
  lambdas = LpVariable.dicts("lambda", range(K), 0, None, cat='Continuous')

  model.setObjective(theta)

  for m in range(M): model.addConstraint(
    sum(lambdas[k] * inputs[m][k] for k in range(K))
    <=
    theta * inputs[m][o]
  )

  for n in range(N): model.addConstraint(
    sum(lambdas[k] * outputs[n][k] for k in range(K))
    >=
    outputs[n][o]
  )

  return model


def find_hcu(alternative: int, inputs_df: DataFrame, outputs_df: DataFrame):
  N, M, K = map(len, [outputs_df.columns, inputs_df.columns, inputs_df])
  o = alternative
  inputs = inputs_df.values.T
  outputs = outputs_df.values.T

  model = create_model(alternative, inputs_df, outputs_df)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  vars = model.variablesDict()

  input_hcus = [
    sum(vars[f'lambda_{k}'].value() * inputs[m][k] for k in range(K))
    for m in range(M)
  ]
  output_hcus = [
    sum(vars[f'lambda_{k}'].value() * outputs[n][k] for k in range(K))
    for n in range(N)
  ]

  input_differences = [
    inputs[m][o] - input_hcus[m]
    for m in range(M)
  ]
  output_differences = [
    output_hcus[n] - outputs[n][o]
    for n in range(N)
  ]

  return [
    *input_hcus,
    *input_differences,
    *output_hcus,
    *output_differences
  ]


rows = [find_hcu(k, inputs_df, outputs_df) for k in range(K)]
columns = [
  *inputs_df.columns.map(lambda x: f'HCU-{x}'),
  *inputs_df.columns.map(lambda x: f'D-{x}'),
  *outputs_df.columns.map(lambda x: f'HCU-{x}'),
  *outputs_df.columns.map(lambda x: f'D-{x}')
]
hcu_results = DataFrame(rows, index=inputs_df.index, columns=columns).round(3)
hcu_results

,HCU-i1,HCU-i2,HCU-i3,HCU-i4,D-i1,D-i2,D-i3,D-i4,HCU-o1,HCU-o2,D-o1,D-o2
airport,,,,,,,,,,,,
WAW,10.500,36.000,129.400,7.000,0.000,0.000,0.000,0.000,9.500,129.700,0.000,0.000
KRK,3.100,19.000,31.600,7.900,0.000,0.000,0.000,0.000,2.900,31.300,0.000,0.000
KAT,2.128,18.919,33.935,4.396,1.472,13.081,23.465,6.104,2.400,29.670,0.000,8.570
WRO,1.500,12.000,18.000,3.000,0.000,0.000,0.000,0.000,1.500,18.800,0.000,0.000
POZ,1.200,7.998,19.195,1.928,0.300,2.002,4.805,2.072,1.300,16.200,0.000,0.000
LCJ,0.180,2.783,7.201,0.475,0.420,9.217,16.799,3.425,0.344,4.200,0.044,0.000
GDN,1.000,15.000,42.900,2.500,0.000,0.000,0.000,0.000,2.000,23.600,0.000,0.000
SZZ,0.190,2.708,6.959,0.465,0.510,7.292,18.741,1.435,0.339,4.200,0.039,-0.000
BZG,0.300,6.000,3.400,1.200,0.000,0.000,0.000,0.000,0.300,6.200,0.000,0.000


## Superefektywność

Do wyznaczenia superefektywności należy z modelu zdjąć ograniczenie efektywności dla kolejnych kryteriów.

\begin{align*}
\max & \sum_{n=1}^{N}\mu_n\cdot y_{no} & \\
s.t. & \sum\limits_{m=1}^{M}\nu_m\cdot x_{mo}=1  &  \\
 & \sum\limits_{n=1}^{N}\mu_n\cdot y_{nk}\leq\sum\limits_{m=1}^{M}\nu_n\cdot x_{mk}, & k=1,2,...,K,\;k\neq o \\
 & \mu_n\nu_m\geq0, & m=1,2,...,M\;, n=1,2,...,N \\
\end{align*}



In [104]:
import pulp
from pulp import LpVariable, LpProblem, LpMaximize


def create_model(alternative_nr: int, inputs_df: DataFrame, outputs_df: DataFrame):
  N, M, K = map(len, [outputs_df.columns, inputs_df.columns, inputs_df])
  o = alternative_nr

  model = LpProblem("CCR-input", LpMaximize)

  input_weights = LpVariable.dicts("input_weight", range(M), 0, None, cat='Continuous')
  output_weights = LpVariable.dicts("output_weight", range(N), 0, None, cat='Continuous')

  inputs = inputs_df.values.T
  outputs = outputs_df.values.T

  model.setObjective(sum(
    output_weights[n] * outputs[n][o] for n in range(N)
  ))

  model.addConstraint(sum(
    input_weights[m] * inputs[m][o] for m in range(M)
  ) == 1)

  for k in filter(lambda k: k != o, range(K)):
    model.addConstraint(
      sum(output_weights[n] * outputs[n][k] for n in range(N))
      <=
      sum(input_weights[m] * inputs[m][k] for m in range(M))
    )

  return model


def find_super_efficiency(alternative: int, inputs_df: DataFrame, outputs_df: DataFrame):
  model = create_model(alternative, inputs_df, outputs_df)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  return model.objective.value()


super_efficiency_results = DataFrame(
  [find_super_efficiency(k, inputs_df, outputs_df) for k in range(K)],
  index=inputs_df.index,
  columns=['super-efficiency']
).round(3)

super_efficiency_results

,super-efficiency
airport,
WAW,2.278
KRK,1.124
KAT,0.591
WRO,1.040
POZ,0.800
LCJ,0.300
GDN,2.000
SZZ,0.271
BZG,1.746


## Efektywność krzyżowa

In [105]:
def find_cross_efficiency(inputs_df: DataFrame, outputs_df: DataFrame) -> DataFrame:
  weights = [find_efficiency_weights(alternative, inputs_df, outputs_df) for alternative in range(len(inputs_df))]
  inputs = inputs_df.values.T
  outputs = outputs_df.values.T

  cross_efficiency = DataFrame([[
    sum(weights[j][f'output_weight_{n}'].varValue * outputs[n][i] for n in range(len(outputs)))
    /
    sum(weights[j][f'input_weight_{m}'].varValue * inputs[m][i] for m in range(len(inputs)))
    for j in range(len(inputs_df))
  ] for i in range(len(inputs_df))
  ],
    index=inputs_df.index,
    columns=inputs_df.index
  )

  cross_efficiency['cross-efficiency'] = cross_efficiency.mean(axis=1)

  return cross_efficiency.round(3)


cross_efficiency_results = find_cross_efficiency(inputs_df, outputs_df)
cross_efficiency_results

airport,WAW,KRK,KAT,WRO,POZ,LCJ,GDN,SZZ,BZG,RZE,IEG,cross-efficiency
airport,,,,,,,,,,,,
WAW,2.278,1.000,0.913,1.000,1.000,0.595,0.905,1.000,0.960,0.903,0.523,1.007
KRK,1.000,1.124,1.000,1.000,1.000,0.491,0.935,0.755,0.948,0.996,0.428,0.880
KAT,0.407,0.519,0.591,0.579,0.563,0.278,0.667,0.371,0.352,0.591,0.248,0.470
WRO,0.980,1.000,1.000,1.040,1.000,0.605,1.000,0.856,1.000,1.000,0.531,0.910
POZ,0.986,0.704,0.774,0.771,0.800,0.512,0.867,0.737,0.646,0.770,0.458,0.729
LCJ,0.216,0.158,0.259,0.241,0.255,0.300,0.500,0.273,0.168,0.261,0.297,0.266
GDN,1.000,0.620,1.000,1.000,1.000,1.000,2.000,1.000,0.527,1.000,1.000,1.013
SZZ,0.266,0.153,0.238,0.239,0.243,0.261,0.429,0.271,0.156,0.238,0.254,0.250
BZG,0.653,0.792,0.972,1.000,0.909,1.000,1.000,1.000,1.746,1.000,0.876,0.995


## Rozkład efektywności

In [125]:
import numpy as np


def find_efficiency_index(inputs_df: DataFrame, outputs_df: DataFrame, samples_df: DataFrame) -> DataFrame:
  M, S, K = map(len, [inputs_df.columns, samples_df, outputs_df])
  inputs = inputs_df.values
  outputs = outputs_df.values
  samples = samples_df.values

  input_weights = samples[:, :M]
  output_weights = samples[:, M:]

  efficiency_scores = DataFrame([[
    sum(output_weights[s] * outputs[k]) / sum(input_weights[s] * inputs[k])
    for s in range(S)
  ] for k in range(K)],
    index=inputs_df.index,
    columns=samples_df.index
  )
  efficiency_scores /= max(efficiency_scores.max())

  bins = np.arange(0, 1.1, 0.2).round(1)

  labels = [f'{bins[i]}-{bins[i + 1]}' for i in range(len(bins) - 1)]

  indices = DataFrame([
    pd.cut(efficiency_scores.loc[index], bins=bins, labels=labels, right=False).value_counts()
    for index in efficiency_scores.index.values
  ])

  indices = DataFrame([
    indices.loc[index] / indices.loc[index].sum()
    for index in indices.index.values
  ])

  indices['index-efficiency'] = efficiency_scores.mean(axis=1)
  return indices


efficiency_index_results = find_efficiency_index(inputs_df, outputs_df, samples_df).round(3)
efficiency_index_results

,0.2-0.4,0.0-0.2,0.4-0.6,0.6-0.8,0.8-1.0,index-efficiency
WAW,0.360,0.330,0.210,0.09,0.01,0.318
KRK,0.420,0.500,0.070,0.01,0.00,0.212
KAT,0.050,0.950,0.000,0.00,0.00,0.090
WRO,0.400,0.480,0.100,0.02,0.00,0.231
POZ,0.360,0.590,0.050,0.00,0.00,0.177
LCJ,0.000,1.000,0.000,0.00,0.00,0.046
GDN,0.300,0.630,0.060,0.01,0.00,0.183
SZZ,0.000,1.000,0.000,0.00,0.00,0.050
BZG,0.293,0.535,0.141,0.03,0.00,0.242
RZE,0.020,0.980,0.000,0.00,0.00,0.073


## Ranking jednostek


In [107]:
def sort_by(frame: DataFrame, column: str) -> DataFrame:
  return frame.sort_values(by=column, ascending=False)

Względem efektywności

In [108]:
sort_by(efficiency_results, 'efficiency')

,efficiency
airport,
WAW,1.000
KRK,1.000
WRO,1.000
GDN,1.000
BZG,1.000
POZ,0.800
KAT,0.591
RZE,0.409
LCJ,0.300


Względem superefektywności

In [109]:
sort_by(super_efficiency_results, 'super-efficiency')

,super-efficiency
airport,
WAW,2.278
GDN,2.000
BZG,1.746
KRK,1.124
WRO,1.040
POZ,0.800
KAT,0.591
RZE,0.409
LCJ,0.300


Względem efektywności krzyżowej

In [110]:
sort_by(cross_efficiency_results, 'cross-efficiency')

airport,WAW,KRK,KAT,WRO,POZ,LCJ,GDN,SZZ,BZG,RZE,IEG,cross-efficiency
airport,,,,,,,,,,,,
GDN,1.000,0.620,1.000,1.000,1.000,1.000,2.000,1.000,0.527,1.000,1.000,1.013
WAW,2.278,1.000,0.913,1.000,1.000,0.595,0.905,1.000,0.960,0.903,0.523,1.007
BZG,0.653,0.792,0.972,1.000,0.909,1.000,1.000,1.000,1.746,1.000,0.876,0.995
WRO,0.980,1.000,1.000,1.040,1.000,0.605,1.000,0.856,1.000,1.000,0.531,0.910
KRK,1.000,1.124,1.000,1.000,1.000,0.491,0.935,0.755,0.948,0.996,0.428,0.880
POZ,0.986,0.704,0.774,0.771,0.800,0.512,0.867,0.737,0.646,0.770,0.458,0.729
KAT,0.407,0.519,0.591,0.579,0.563,0.278,0.667,0.371,0.352,0.591,0.248,0.470
RZE,0.352,0.332,0.409,0.382,0.403,0.273,0.500,0.346,0.297,0.409,0.247,0.359
LCJ,0.216,0.158,0.259,0.241,0.255,0.300,0.500,0.273,0.168,0.261,0.297,0.266


Względem rozkładu efektywności

In [111]:
sort_by(efficiency_index_results, 'index-efficiency')

,0.2-0.4,0.0-0.2,0.4-0.6,0.6-0.8,0.8-1.0,index-efficiency
WAW,0.36,0.33,0.21,0.09,0.01,0.318
BZG,0.29,0.53,0.14,0.03,0.01,0.242
WRO,0.40,0.48,0.10,0.02,0.00,0.231
KRK,0.42,0.50,0.07,0.01,0.00,0.212
GDN,0.30,0.63,0.06,0.01,0.00,0.183
POZ,0.36,0.59,0.05,0.00,0.00,0.177
KAT,0.05,0.95,0.00,0.00,0.00,0.090
RZE,0.02,0.98,0.00,0.00,0.00,0.073
SZZ,0.00,1.00,0.00,0.00,0.00,0.050
LCJ,0.00,1.00,0.00,0.00,0.00,0.046
